<a href="https://colab.research.google.com/github/marioabduh/portfolio/blob/main/CSV_Splitter_from_GDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Google Colab for the first run

In [ ]:
# Install necessary modules

!pip install kora
from kora.xattr import get_id

!pip install -U -q PyDrive2
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
gdrive = GoogleDrive(gauth)

from google.colab import drive
drive.mount('/content/drive/')

# Imports necessary nmodules
import pandas as pd
import random
import string
import shutil
import os



print('Setup Complete!')

# Specify Google Drive folder location and get folderID

In [ ]:
# Specify the big files folder path in your Google Drive
big_files_folder = '/content/drive/My Drive/Portfolio/Big Files'
file_list = os.listdir(big_files_folder)
for file_name in file_list:
    try:
        file_path = os.path.join(big_files_folder, file_name)
        os.remove(file_path)
    except Exception as e:
        print(f"Error deleting {file_name}: {e}")

# Get folderID for big files folder
folder_id = get_id(big_files_folder)


# Specify the small files folder path in your Google Drive
small_files_folder = '/content/drive/My Drive/Portfolio/Small Files'
file_list = os.listdir(small_files_folder)
for file_name in file_list:
    try:
        file_path = os.path.join(small_files_folder, file_name)
        os.remove(file_path)
    except Exception as e:
        print(f"Error deleting {file_name}: {e}")

# Create a random char, save it as csv, and move it to a Big Files folder in Google Drive.

In [ ]:
def generate_random_string(length):
    characters = string.ascii_letters + string.digits
    return ''.join(random.choice(characters) for _ in range(length))

num_lines = 1000000
for i in range(2):
  random_strings = [generate_random_string(10) for _ in range(num_lines)]

  # Create a DataFrame with the random strings
  df = pd.DataFrame({'RandomString': random_strings})

  # Save the DataFrame to a CSV file
  df.to_csv(f'random_strings-{i}.csv', index=False)

  # Specify the source path (where the CSV file is currently)
  source_path = f'random_strings-{i}.csv'

  # Move the file
  shutil.move(source_path, big_files_folder)

# Get CSV files from a folder and join it in 1 csv files

In [ ]:
# import and merge csv content from drive folder to pandas
query = f"'{folder_id}' in parents and trashed=false and fileExtension='csv'"
list_df = gdrive.ListFile({'q': query}).GetList()
list_file_dr = []
list_title_df = []
for file_df in list_df:
    list_file_dr.append(file_df['id'])
    list_title_df.append(file_df['title'])

df_df = pd.DataFrame()

for id, title in zip(list_file_dr, list_title_df):
    each_file_df = gdrive.CreateFile({'id': id})
    each_file_df.GetContentFile(title)
    df_each_file_df = pd.read_csv(title)
    print(title, len(df_each_file_df))
    # df_df = df_df.append(df_each_file_df, ignore_index=True)
    df_df = pd.concat([df_df,df_each_file_df], ignore_index = True)


df_df.info()
df_df.head()

# Split the csv file into smaller file containing custom row length

In [ ]:
chunk_size = 100000
list_df_df = [df_df[i:i+chunk_size] for i in range(0,len(df_df),chunk_size)]
for i, chunk in enumerate(list_df_df):
    chunk.to_csv(f'/content/drive/My Drive/Portfolio/Small Files/file_{i}.csv', header=None, index=None)

In [ ]:
print("Done!")